In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/thesis/Finaldf.csv
/kaggle/input/news-articles/Articles.csv


In [ ]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
np.random.seed(42)
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import nltk, string, re, spacy,unicodedata, random
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


CPU times: user 5.2 s, sys: 1.14 s, total: 6.34 s
Wall time: 11 s


In [ ]:
import pandas as pd
data = pd.read_csv("/kaggle/input/thesis/Finaldf.csv")
print(f'Total number of Documents: {len(data)}')

Total number of Documents: 7300


In [ ]:
import pandas as pd

# Assuming your dataset is stored in a DataFrame called 'df'
# Replace 'clean text' with the actual column name in your dataset
duplicates = data[data.duplicated(subset='text', keep=False)]

# 'duplicates' now contains all rows where the 'clean text' column has duplicate values
# You can inspect these entries to understand why they are duplicated

# To print the count of duplicated rows:
print(f"Number of duplicated rows: {len(duplicates)}")

Number of duplicated rows: 0


In [ ]:
data = data[['text','label']]

In [ ]:
data = data.rename(columns={0:'text',1:'label'})

In [ ]:
total_categories = data['label'].nunique()
print("Total number of categories:", total_categories)

Total number of categories: 6


In [ ]:
# Assuming 'data' is your DataFrame with columns renamed
category_names = data['label'].unique()
print("Category names:", category_names)

Category names: ['Software Engineering' 'DBMS' 'Data Communication' 'Cyber Security '
 'Operating System' 'Cyber Security']


In [ ]:
# Replace 'Cyber Security' and 'Cyber Security ' with 'Cyber Security'
data['label'] = data['label'].replace(['Cyber Security ', 'Cyber Security  '], 'Cyber Security')

# Replace 'Operating System' and 'operating System' with 'Operating System'
data['label'] = data['label'].replace(['Operating System', 'operating System'], 'Operating System')

In [ ]:
# Assuming 'data' is your DataFrame with the 'label' column containing category names
unique_classes = data['label'].unique()
print("Unique classes:", unique_classes)

Unique classes: ['Software Engineering' 'DBMS' 'Data Communication' 'Cyber Security'
 'Operating System']


In [ ]:
def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\u00C0-\u017F"          #latin
                           u"\u2000-\u206F"          #generalPunctuations

                           "]+", flags=re.UNICODE)
    #english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    #latin_pattern=re.compile('[A-Za-z\u00C0-\u00D6\u00D8-\u00f6\u00f8-\u00ff\s]*',)

    text=emoji_pattern.sub(r'', text)
    #text=english_pattern.sub(r'', text)
    text = re.sub(r'₀|₁|₂|₃|₄|₅|₆|₇|₈|₉', '', text)
    #text = re.sub(r'⁰|¹|²|³|⁴|⁵|⁶|⁷|⁸|⁹|⁻|⁺|ⁿ|ᵗ', '', text)

    return text

def remove_punctuations(my_str):
    # define punctuation
    punctuations = '''````£|¢|Ñ+-*/=৳০১২৩৪৫৬৭৮৯012–34567•89।!()-−[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰θ→°π∈∝α≤≥Δβε≠�￰৷￰'''

    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct



def joining(text):
    out=' '.join(text)
    return out

def preprocessing(text):
    out=remove_punctuations(replace_strings(text))
    return out


In [ ]:
data['cleaned'] = data['text'].apply(lambda x: preprocessing(str(x)))

In [ ]:
sample_data = range(4000, 4005)
for i in sample_data:
    print(data['cleaned'][i])

গ্রে হ্যাট হ্যাকাররা  আবার তা নিজের স্বার্থের জন্যও ব্যবহার করতে পারে
ব্ল্যাক হ্যাট হ্যাকার Black Hat Hacker এই ধরনের হ্যাকাররা সবচেয়ে ভয়ংকর প্রকৃতির হ্যাকার 
ব্ল্যাক হ্যাট হ্যাকার  কোনো একটি সিকিউরিটি সিস্টেমের ত্রুটিগুলো বের করলে দ্রুত ঐ ত্রুটিকে নিজের স্বার্থে কাজে লাগায় 
ব্ল্যাক হ্যাট হ্যাকার  এরা সিস্টেম নষ্ট করে বিভিন্ন ভাইরাস ছড়িয়ে দেয় 
ব্ল্যাক হ্যাট হ্যাকার  তারা ভবিষ্যতে নিজে আবার যেন ঢুকতে পারে সে পথ রাখে 


In [ ]:
stop =["অতএব","অথচ","অথবা","অনুযায়ী","অনেক","অনেকে","অনেকেই","অন্তত","অন্য","অবধি","অবশ্য","অর্থাত","আই","আগামী","আগে","আগেই","আছে","আজ","আদ্যভাগে","আপনার","আপনি","আবার","আমরা","আমাকে","আমাদের","আমার","আমি","আর","আরও","ই","ইত্যাদি","ইহা","উচিত","উত্তর","উনি","উপর","উপরে","এ","এঁদের","এঁরা","এই","একই","একটি","একবার","একে","এক্","এখন","এখনও","এখানে","এখানেই","এটা","এটাই","এটি","এত","এতটাই","এতে","এদের","এব","এবং","এবার","এমন","এমনকী","এমনি","এর","এরা","এল","এস","এসে","ঐ","ও","ওঁদের","ওঁর","ওঁরা","ওই","ওকে","ওখানে","ওদের","ওর","ওরা","কখনও","কত","কবে","কমনে","কয়েক","কয়েকটি","করছে","করছেন","করতে","করবে","করবেন","করলে","করলেন","করা","করাই","করায়","করার","করি","করিতে","করিয়া","করিয়ে","করে","করেই","করেছিলেন","করেছে","করেছেন","করেন","কাউকে","কাছ","কাছে","কাজ","কাজে","কারও","কারণ","কি","কিংবা","কিছু","কিছুই","কিন্তু","কী","কে","কেউ","কেউই","কেখা","কেন","কোটি","কোন","কোনও","কোনো","ক্ষেত্রে","কয়েক","খুব","গিয়ে","গিয়েছে","গিয়ে","গুলি","গেছে","গেল","গেলে","গোটা","চলে","চান","চায়","চার","চালু","চেয়ে","চেষ্টা","ছাড়া","ছাড়াও","ছিল","ছিলেন","জন","জনকে","জনের","জন্য","জন্যওজে","জানতে","জানা","জানানো","জানায়","জানিয়ে","জানিয়েছে","জে","জ্নজন","টি","ঠিক","তখন","তত","তথা","তবু","তবে","তা","তাঁকে","তাঁদের","তাঁর","তাঁরা","তাঁাহারা","তাই","তাও","তাকে","তাতে","তাদের","তার","তারপর","তারা","তারৈ","তাহলে","তাহা","তাহাতে","তাহার","তিনঐ","তিনি","তিনিও","তুমি","তুলে","তেমন","তো","তোমার","থাকবে","থাকবেন","থাকা","থাকায়","থাকে","থাকেন","থেকে","থেকেই","থেকেও","দিকে","দিতে","দিন","দিয়ে","দিয়েছে","দিয়েছেন","দিলেন","দু","দুই","দুটি","দুটো","দেওয়া","দেওয়ার","দেওয়া","দেখতে","দেখা","দেখে","দেন","দেয়","দ্বারা","ধরা","ধরে","ধামার","নতুন","নয়","না","নাই","নাকি","নাগাদ","নানা","নিজে","নিজেই","নিজেদের","নিজের","নিতে","নিয়ে","নিয়ে","নেই","নেওয়া","নেওয়ার","নেওয়া","নয়","পক্ষে","পর","পরে","পরেই","পরেও","পর্যন্ত","পাওয়া","পাচ","পারি","পারে","পারেন","পি","পেয়ে","পেয়্র্","প্রতি","প্রথম","প্রভৃতি","প্রযন্ত","প্রাথমিক","প্রায়","প্রায়","ফলে","ফিরে","ফের","বক্তব্য","বদলে","বন","বরং","বলতে","বলল","বললেন","বলা","বলে","বলেছেন","বলেন","বসে","বহু","বা","বাদে","বার","বি","বিনা","বিভিন্ন","বিশেষ","বিষয়টি","বেশ","বেশি","ব্যবহার","ব্যাপারে","ভাবে","ভাবেই","মতো","মতোই","মধ্যভাগে","মধ্যে","মধ্যেই","মধ্যেও","মনে","মাত্র","মাধ্যমে","মোট","মোটেই","যখন","যত","যতটা","যথেষ্ট","যদি","যদিও","যা","যাঁর","যাঁরা","যাওয়া","যাওয়ার","যাওয়া","যাকে","যাচ্ছে","যাতে","যাদের","যান","যাবে","যায়","যার","যারা","যিনি","যে","যেখানে","যেতে","যেন","যেমন","র","রকম","রয়েছে","রাখা","রেখে","লক্ষ","শুধু","শুরু","সঙ্গে","সঙ্গেও","সব","সবার","সমস্ত","সম্প্রতি","সহ","সহিত","সাথে","সাধারণ","সামনে","সি","সুতরাং","সে","সেই","সেখান","সেখানে","সেটা","সেটাই","সেটাও","সেটি","স্পষ্ট","স্বয়ং","হইতে","হইবে","হইয়া","হওয়া","হওয়ায়","হওয়ার","হচ্ছে","হত","হতে","হতেই","হন","হবে","হবেন","হয়","হয়তো","হয়নি","হয়ে","হয়েই","হয়েছিল","হয়েছে","হয়েছেন","হল","হলে","হলেই","হলেও","হলো","হাজার","হিসাবে","হৈলে","হোক","হয়"]
display(stop)

['অতএব',
 'অথচ',
 'অথবা',
 'অনুযায়ী',
 'অনেক',
 'অনেকে',
 'অনেকেই',
 'অন্তত',
 'অন্য',
 'অবধি',
 'অবশ্য',
 'অর্থাত',
 'আই',
 'আগামী',
 'আগে',
 'আগেই',
 'আছে',
 'আজ',
 'আদ্যভাগে',
 'আপনার',
 'আপনি',
 'আবার',
 'আমরা',
 'আমাকে',
 'আমাদের',
 'আমার',
 'আমি',
 'আর',
 'আরও',
 'ই',
 'ইত্যাদি',
 'ইহা',
 'উচিত',
 'উত্তর',
 'উনি',
 'উপর',
 'উপরে',
 'এ',
 'এঁদের',
 'এঁরা',
 'এই',
 'একই',
 'একটি',
 'একবার',
 'একে',
 'এক্',
 'এখন',
 'এখনও',
 'এখানে',
 'এখানেই',
 'এটা',
 'এটাই',
 'এটি',
 'এত',
 'এতটাই',
 'এতে',
 'এদের',
 'এব',
 'এবং',
 'এবার',
 'এমন',
 'এমনকী',
 'এমনি',
 'এর',
 'এরা',
 'এল',
 'এস',
 'এসে',
 'ঐ',
 'ও',
 'ওঁদের',
 'ওঁর',
 'ওঁরা',
 'ওই',
 'ওকে',
 'ওখানে',
 'ওদের',
 'ওর',
 'ওরা',
 'কখনও',
 'কত',
 'কবে',
 'কমনে',
 'কয়েক',
 'কয়েকটি',
 'করছে',
 'করছেন',
 'করতে',
 'করবে',
 'করবেন',
 'করলে',
 'করলেন',
 'করা',
 'করাই',
 'করায়',
 'করার',
 'করি',
 'করিতে',
 'করিয়া',
 'করিয়ে',
 'করে',
 'করেই',
 'করেছিলেন',
 'করেছে',
 'করেছেন',
 'করেন',
 'কাউকে',
 'কাছ',
 'কাছে',
 'কাজ',
 'কাজে',
 'কারও',
 '

In [ ]:
def stopwordRemoval(text):
    x=str(text)
    l=x.split()

    stm=[elem for elem in l if elem not in stop]

    out=' '.join(stm)

    return str(out)

In [ ]:
# Apply the stopwordRemoval function to the 'clean text' column
data['cleaned'] = data['cleaned'].apply(lambda x: stopwordRemoval(str(x)))

# Iterate through the first 1000 rows and print the 'clean text' column values
sample_data = range(0, 5)
for i in sample_data:
    print(data['cleaned'][i])

সফটওয়্যার ইঞ্জিনিয়ারিং দুইটি শব্দ সফটওয়্যার ইঞ্জিনিয়ারিংএর সমন্বয়ে গঠিত
সফটওয়্যার প্রোগ্রামের সমষ্টি
প্রোগ্রাম নির্ধারিত সমস্যা সমাধানের উদ্দেশ্যে তৈরি সফটওয়্যার প্রোডাক্ট
অপরদিকে ইঞ্জিনিয়ারিং সকল নীতিমালা মেথড সমূহ প্রোডাক্টটি Software ফিনিশিং প্রোডাক্ট সেগুলোর সুশৃঙ্খল প্রয়োগ করাকে বুঝায়
সফটওয়্যার ইঞ্জিনিয়ারিং বুঝায়


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training (80%) and temporary data (20%)
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)

# Further split the temporary data into validation (50%) and test data (50%)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Print the shapes of the resulting datasets
print("Training data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)
print("Testing data shape:", test_data.shape)

Training data shape: (5840, 3)
Validation data shape: (730, 3)
Testing data shape: (730, 3)


In [ ]:
train_data['enc_label'] = train_data['label'].replace({'Software Engineering':0, 'DBMS':1, 'Data Communication':2 ,'Cyber Security':3,
 'Operating System':4})
val_data['enc_label'] = val_data['label'].replace({'Software Engineering':0, 'DBMS':1, 'Data Communication':2 ,'Cyber Security':3,
 'Operating System':4})

In [ ]:
train_data['cleaned'].tolist()

['টপ ডাউন ডিজাইনে সংখ্যক বৈশিষ্ট্য ভিত্তিতে এটিকে একাধিক সাব সিস্টেম উপাদানে ডিকম্পোজ',
 'সফ্টওয়্যার প্রকল্পের শুরুতে ব্যবসার প্রয়োজন প্রায়ই কথোপকথনের অংশ অনানুষ্ঠানিকভাবে প্রকাশ',
 'অপারেটিং সিস্টেমের অন্তর্গত system programসমূহকে ক্যাটাগরিতে ভাগ যায়',
 'Accelerometer মোবাইল ডিভাইসকে গ্রাউন্ডের সাপেক্ষে ওরিয়েন্টেশন ডিটেক্ট অন্যান্য নির্দিষ্ট ফোর্সসমূহকে ডিটেক্ট',
 'perfect testing সবচেয়ে কাংখিত মেথড Exhaustive test',
 'তৃতীয়ত অপারেটিং সিস্টেম সিস্টেমের সকল উপাদান Components সংযোগ রক্ষা',
 'সফটওয়্যার ভেলিডেশন প্রক্রিয়া ইউজারের চাহিদা মোতাবেক সফটওয়্যারটি তৈরি হয়েছে পরীক্ষনের ব্যবহৃত',
 'সফ্টওয়্যার ডিজাইনের ল্যান্ডমার্ক পেপারে প্রতিটি মডিউলের স্বাধীনতাকে উন্নত কৌশলগুলিকে পরিমার্জন ইঙ্গিত',
 'Tape Diskএর গতি বাইট',
 'ট্রোজান হর্স এক ধরনের ক্ষতিকর Program Software ইন্টারনেট অ্যাক্টিভিটিস ডেটা নেটওয়ার্কের ক্ষতি কম্পিউটার সিস্টেমের সুরক্ষা বিঘ্নিত',
 'ফার্স্ট ফিট পদ্ধতিতে প্রসেস পর্যাপ্ত সাইজের ফ্রি পার্টিশন পাবে সেখানেই প্রসেসটি লোড',
 'Multiprocessor systemএর খরচ Multiple single proces

In [ ]:
train_data['enc_label'].value_counts()

3    1273
2    1239
0    1150
1    1096
4    1082
Name: enc_label, dtype: int64

In [ ]:
val_data['enc_label'].value_counts()

3    159
2    147
0    144
1    141
4    139
Name: enc_label, dtype: int64

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [ ]:
train_data = train_data.rename(columns={'label_Software Engineering':'Software Engineering','label_DBMS':'DBMS',
                                        'label_Data Communication':'Data Communication','label_Cyber Security':'Cyber Security','label_Operating System':'Operating System'})

train_data

,text,label,cleaned,enc_label
730,টপ ডাউন ডিজাইনে কিছু সংখ্যক বৈশিষ্ট্য এর ভিত্ত...,Software Engineering,টপ ডাউন ডিজাইনে সংখ্যক বৈশিষ্ট্য ভিত্তিতে এটিক...,0
5780,"একটি সফ্টওয়্যার প্রকল্পের শুরুতে, ব্যবসার প্র...",Software Engineering,সফ্টওয়্যার প্রকল্পের শুরুতে ব্যবসার প্রয়োজন ...,0
4685,অপারেটিং সিস্টেমের অন্তর্গত system program-সমূ...,Operating System,অপারেটিং সিস্টেমের অন্তর্গত system programসমূহ...,4
5041,Accelerometer মোবাইল ডিভাইসকে গ্রাউন্ডের সাপেক...,Operating System,Accelerometer মোবাইল ডিভাইসকে গ্রাউন্ডের সাপেক...,4
325,একটি perfect testing এর সবচেয়ে কাংখিত মেথড হলো...,Software Engineering,perfect testing সবচেয়ে কাংখিত মেথড Exhaustive ...,0
...,...,...,...,...
5191,স্বয়ংক্রিয়ভাবে এক জব থেকে অন্য জবে যাওয়ার উপর ...,Operating System,স্বয়ংক্রিয়ভাবে এক জব জবে যাওয়ার ভিত্তি অপারেটি...,4
5226,সার্ভার প্রথমে ক্লায়েন্টের নিকট হতে নির্দিষ্ট ...,Operating System,সার্ভার প্রথমে ক্লায়েন্টের নিকট নির্দিষ্ট Part...,4
5390,যদি কোনো System-এ Deadlock prevention বা Deadl...,Operating System,Systemএ Deadlock prevention Deadlock avoidance...,4
860,কম্পিউটার প্রোগ্রামিং ভাষা প্রয়োগের মাধ্যমে সফ...,Software Engineering,কম্পিউটার প্রোগ্রামিং ভাষা প্রয়োগের সফটওয়্যার ...,0


In [ ]:
def char_length(row):
    return len(row.split())

In [ ]:
lables = ['Software Engineering' 'DBMS' 'Data Communication' 'Cyber Security'
 'Operating System']

# Count number of unique words in each set
def dataset_statistics(dataset):
  for l in lables:
    word_list = [word.strip() for t in list(dataset[dataset[l]==1.0].cleaned) for word in t.strip().split()]
    print(f"\nClass Name:--> {l} ")
    print("Number of Words:{}".format(len(word_list)))
    print("Number of Unique Words:{}".format(len(np.unique(word_list))))
    print("Maximum post length: {}".format(max(dataset[dataset[l]==1.0].cleaned.apply(char_length))))
    print("Average Words in texts: {}".format(np.mean(dataset[dataset[l]==1.0].cleaned.apply(char_length))))

In [ ]:
# Class Distribution
class_distribution = train_data['label'].value_counts()

# Text Length Distribution
train_data['text_length'] = train_data['cleaned'].apply(lambda x: len(x.split()))
text_length_stats = train_data['text_length'].describe()

print("Class Distribution:")
print(class_distribution)
print("\nText Length Distribution:")
print(text_length_stats)

Class Distribution:
Cyber Security          1273
Data Communication      1239
Software Engineering    1150
DBMS                    1096
Operating System        1082
Name: label, dtype: int64

Text Length Distribution:
count    5840.000000
mean        9.955993
std         4.812670
min         1.000000
25%         7.000000
50%         9.000000
75%        12.000000
max        69.000000
Name: text_length, dtype: float64


In [ ]:
class_statistics = {}

# Iterate over each unique class
for class_name in train_data['label'].unique():
    # Filter the training data for the current class
    class_data = train_data[train_data['label'] == class_name]

    # Number of Words
    total_words = class_data['cleaned'].apply(lambda x: len(x.split())).sum()

    # Number of Unique Words
    unique_words = len(set(" ".join(class_data['cleaned']).split()))

    # Maximum Post Length
    max_length = class_data['cleaned'].apply(lambda x: len(x.split())).max()

    # Average Words in Texts
    average_length = class_data['cleaned'].apply(lambda x: len(x.split())).mean()

    # Store the statistics for the current class
    class_statistics[class_name] = {
        'Number of Words': total_words,
        'Number of Unique Words': unique_words,
        'Maximum post length': max_length,
        'Average Words in texts': average_length
    }

# Print the statistics for each class
for class_name, stats in class_statistics.items():
    print("Class Name:", class_name)
    for stat_name, value in stats.items():
        print(f"{stat_name}: {value}")
    print()

Class Name: Software Engineering
Number of Words: 12080
Number of Unique Words: 3401
Maximum post length: 57
Average Words in texts: 10.504347826086956

Class Name: Operating System
Number of Words: 10435
Number of Unique Words: 3268
Maximum post length: 34
Average Words in texts: 9.644177449168208

Class Name: Cyber Security
Number of Words: 12662
Number of Unique Words: 3934
Maximum post length: 59
Average Words in texts: 9.946582875098192

Class Name: DBMS
Number of Words: 10883
Number of Unique Words: 3095
Maximum post length: 69
Average Words in texts: 9.929744525547445

Class Name: Data Communication
Number of Words: 12083
Number of Unique Words: 3802
Maximum post length: 33
Average Words in texts: 9.752219531880549



In [ ]:
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 48.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 59.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 37.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ktrain: filename=ktrain-0.41.3-py3-none-any.whl size=25316979 sha256=ad325227111611de2679884f3df561a3224715e19544bd2901cbde2ce92c6f63
  Stored in directory: /root/.cache/pip/wheels/97/cd/a7/3ef9b6abdab10d64a111bac7f6f1cc4e9ea818c478335fb394
  Created wheel for keras_bert: fil

In [ ]:
import ktrain

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import ktrain
from ktrain import text

In [ ]:
train_data['label'].value_counts()

Cyber Security          1273
Data Communication      1239
Software Engineering    1150
DBMS                    1096
Operating System        1082
Name: label, dtype: int64

In [ ]:
val_data['label'].value_counts()

Cyber Security          159
Data Communication      147
Software Engineering    144
DBMS                    141
Operating System        139
Name: label, dtype: int64

In [ ]:
categories = ['Software Engineering', 'DBMS' ,'Data Communication', 'Cyber Security',
 'Operating System']

In [ ]:
X_train = train_data['cleaned'].tolist()
y_train = train_data['label'].tolist()

X_valid = val_data['cleaned'].tolist()
y_valid = val_data['label'].tolist()

X_test = test_data['cleaned'].tolist()
y_test = test_data['label'].tolist()

In [ ]:
model_name = 'xlm-roberta-base'
trans = text.Transformer(model_name,maxlen=100,class_names=categories)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.76G [00:00<?, ?B/s]

In [ ]:
train = trans.preprocess_train(X_train,y_train)
valid = trans.preprocess_test(X_valid,y_valid)

preprocessing train...
language: bn
train sequence lengths:
	mean : 10
	95percentile : 19
	99percentile : 26


Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 10
	95percentile : 19
	99percentile : 25


In [ ]:
model = trans.get_classifier()

In [ ]:
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=32)

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced'
                                               ,classes = np.unique(train_data['enc_label'])
                                               ,y = train_data['enc_label'])
class_weights = dict(zip(np.unique(train_data['enc_label']), class_weights))

In [ ]:
weight = {i : class_weights[i] for i in range(5)}
weight

{0: 1.0156521739130435,
 1: 1.0656934306569343,
 2: 0.9426957223567393,
 3: 0.9175176747839748,
 4: 1.0794824399260627}

In [ ]:
learner.fit_onecycle(2e-5,10,class_weight=weight)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
183/183 [==============================] - 106s 454ms/step - loss: 1.5910 - accuracy: 0.2608 - val_loss: 1.4383 - val_accuracy: 0.5781
Epoch 2/10
183/183 [==============================] - 80s 437ms/step - loss: 1.1206 - accuracy: 0.6140 - val_loss: 0.6577 - val_accuracy: 0.7658
Epoch 3/10
183/183 [==============================] - 80s 437ms/step - loss: 0.6906 - accuracy: 0.7760 - val_loss: 0.5426 - val_accuracy: 0.8233
Epoch 4/10
183/183 [==============================] - 80s 436ms/step - loss: 0.4963 - accuracy: 0.8406 - val_loss: 0.4651 - val_accuracy: 0.8479
Epoch 5/10
183/183 [==============================] - 80s 436ms/step - loss: 0.4026 - accuracy: 0.8764 - val_loss: 0.4263 - val_accuracy: 0.8767
Epoch 6/10
183/183 [==============================] - 80s 436ms/step - loss: 0.3007 - accuracy: 0.9098 - val_loss: 0.3573 - val_accuracy: 0.8932
Epoch 7/10
183/183 [==============================] - 80s 436ms/st

In [ ]:
learner.validate(class_names=categories)

                      precision    recall  f1-score   support

Software Engineering       0.95      0.86      0.90       159
                DBMS       0.81      0.90      0.85       141
  Data Communication       0.90      0.95      0.93       147
      Cyber Security       0.91      0.89      0.90       139
    Operating System       0.91      0.88      0.90       144

            accuracy                           0.90       730
           macro avg       0.90      0.90      0.90       730
        weighted avg       0.90      0.90      0.90       730



array([[136,   9,   7,   5,   2],
       [  4, 127,   1,   4,   5],
       [  2,   3, 140,   1,   1],
       [  0,   7,   4, 124,   4],
       [  1,  11,   3,   2, 127]])

In [ ]:
# Save the predictor directly in the working directory
save_path = '/kaggle/working/roberta'
ktrain.get_predictor(learner.model, preproc=trans).save(save_path)

In [ ]:
# Import necessary packages
from sklearn.metrics import accuracy_score, classification_report
import ktrain

# Specify the file path for saving and loading the model in the Kaggle working directory
save_path = '/kaggle/working/roberta'

# Load the predictor from the specified path
predictor = ktrain.load_predictor(save_path)

# Assuming you have your test data available as 'X_test' and 'y_test' in your Kaggle environment
# Make predictions on the test data using the loaded predictor
y_pred = predictor.predict(X_test)

# Calculate accuracy by comparing the predicted labels with true labels
accuracy = accuracy_score(y_test, y_pred)
print(f"Test data accuracy: {accuracy:.2%}")

# Generate classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print("Classification Report:")
print(report)

Test data accuracy: 88.36%
Classification Report:
                      precision    recall  f1-score   support

      Cyber Security       0.95      0.90      0.93       152
                DBMS       0.89      0.82      0.85       164
  Data Communication       0.89      0.94      0.91       148
    Operating System       0.82      0.85      0.84       132
Software Engineering       0.86      0.91      0.88       134

            accuracy                           0.88       730
           macro avg       0.88      0.88      0.88       730
        weighted avg       0.88      0.88      0.88       730



In [ ]:
# Import the necessary libraries
from ktrain import text

# Assuming 'learner' is your ktrain learner object
# Get the underlying model from ktrain learner
model = learner.model

# Save the model
model_name = 'xlm_roberta_model'
model_path = f'/kaggle/working/{model_name}'

# Save the model's weights and architecture
model.save_pretrained(model_path)
print(f"Model saved to {model_path}")

Model saved to /kaggle/working/xlm_roberta_model


In [ ]:
model_name = 'google-bert/bert-base-multilingual-cased'
trans = text.Transformer(model_name,maxlen=100,class_names=categories)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

In [ ]:
train = trans.preprocess_train(X_train,y_train)
valid = trans.preprocess_test(X_valid,y_valid)

preprocessing train...
language: bn
train sequence lengths:
	mean : 10
	95percentile : 19
	99percentile : 26


Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 10
	95percentile : 19
	99percentile : 25


In [ ]:
model = trans.get_classifier()

In [ ]:
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=32)

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced'
                                               ,classes = np.unique(train_data['enc_label'])
                                               ,y = train_data['enc_label'])
class_weights = dict(zip(np.unique(train_data['enc_label']), class_weights))

In [ ]:
weight = {i : class_weights[i] for i in range(5)}
weight

{0: 1.0156521739130435,
 1: 1.0656934306569343,
 2: 0.9426957223567393,
 3: 0.9175176747839748,
 4: 1.0794824399260627}

In [ ]:
learner.fit_onecycle(2e-5,10,class_weight=weight)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
183/183 [==============================] - 99s 429ms/step - loss: 1.5728 - accuracy: 0.2854 - val_loss: 1.3286 - val_accuracy: 0.5411
Epoch 2/10
183/183 [==============================] - 75s 412ms/step - loss: 0.9910 - accuracy: 0.6625 - val_loss: 0.6579 - val_accuracy: 0.8027
Epoch 3/10
183/183 [==============================] - 75s 413ms/step - loss: 0.5721 - accuracy: 0.8103 - val_loss: 0.5001 - val_accuracy: 0.8301
Epoch 4/10
183/183 [==============================] - 75s 412ms/step - loss: 0.3892 - accuracy: 0.8781 - val_loss: 0.4793 - val_accuracy: 0.8479
Epoch 5/10
183/183 [==============================] - 75s 412ms/step - loss: 0.3098 - accuracy: 0.8991 - val_loss: 0.4011 - val_accuracy: 0.8808
Epoch 6/10
183/183 [==============================] - 75s 412ms/step - loss: 0.2006 - accuracy: 0.9382 - val_loss: 0.4544 - val_accuracy: 0.8658
Epoch 7/10
183/183 [==============================] - 75s 412ms/ste

In [ ]:
learner.validate(class_names=categories)

                      precision    recall  f1-score   support

Software Engineering       0.93      0.89      0.91       159
                DBMS       0.86      0.90      0.88       141
  Data Communication       0.88      0.93      0.91       147
      Cyber Security       0.92      0.88      0.90       139
    Operating System       0.90      0.89      0.90       144

            accuracy                           0.90       730
           macro avg       0.90      0.90      0.90       730
        weighted avg       0.90      0.90      0.90       730



array([[141,   3,   9,   1,   5],
       [  4, 127,   3,   3,   4],
       [  0,   5, 137,   3,   2],
       [  3,   8,   2, 123,   3],
       [  3,   5,   4,   4, 128]])

In [ ]:
save_path = '/kaggle/working/mbert'
ktrain.get_predictor(learner.model, preproc=trans).save(save_path)

In [ ]:
# Import necessary packages
from sklearn.metrics import accuracy_score, classification_report
import ktrain

# Specify the file path for saving and loading the model in the Kaggle working directory
save_path = '/kaggle/working/mbert'

# Load the predictor from the specified path
predictor = ktrain.load_predictor(save_path)

# Assuming you have your test data available as 'X_test' and 'y_test' in your Kaggle environment
# Make predictions on the test data using the loaded predictor
y_pred = predictor.predict(X_test)

# Calculate accuracy by comparing the predicted labels with true labels
accuracy = accuracy_score(y_test, y_pred)
print(f"Test data accuracy: {accuracy:.2%}")

# Generate classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print("Classification Report:")
print(report)

Test data accuracy: 89.45%
Classification Report:
                      precision    recall  f1-score   support

      Cyber Security       0.92      0.89      0.91       152
                DBMS       0.91      0.85      0.88       164
  Data Communication       0.90      0.91      0.91       148
    Operating System       0.86      0.90      0.88       132
Software Engineering       0.87      0.93      0.90       134

            accuracy                           0.89       730
           macro avg       0.89      0.90      0.89       730
        weighted avg       0.90      0.89      0.89       730



In [ ]:
# Import the necessary libraries
from ktrain import text

# Assuming 'learner' is your ktrain learner object
# Get the underlying model from ktrain learner
model = learner.model

# Save the model
model_name = 'mbert_model'
model_path = f'/kaggle/working/{model_name}'

# Save the model's weights and architecture
model.save_pretrained(model_path)
print(f"Model saved to {model_path}")

Model saved to /kaggle/working/mbert_model


In [ ]:
model_name = 'distilbert-base-multilingual-cased'
trans = text.Transformer(model_name,maxlen=100,class_names=categories)

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/869M [00:00<?, ?B/s]

In [ ]:
train = trans.preprocess_train(X_train,y_train)
valid = trans.preprocess_test(X_valid,y_valid)

preprocessing train...
language: bn
train sequence lengths:
	mean : 10
	95percentile : 19
	99percentile : 26


Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 10
	95percentile : 19
	99percentile : 25


In [ ]:
model = trans.get_classifier()

In [ ]:
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=32)

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced'
                                               ,classes = np.unique(train_data['enc_label'])
                                               ,y = train_data['enc_label'])
class_weights = dict(zip(np.unique(train_data['enc_label']), class_weights))

In [ ]:
weight = {i : class_weights[i] for i in range(5)}
weight

{0: 1.0156521739130435,
 1: 1.0656934306569343,
 2: 0.9426957223567393,
 3: 0.9175176747839748,
 4: 1.0794824399260627}

In [ ]:
learner.fit_onecycle(2e-5,10,class_weight=weight)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
183/183 [==============================] - 53s 231ms/step - loss: 1.5791 - accuracy: 0.2705 - val_loss: 1.4243 - val_accuracy: 0.4151
Epoch 2/10
183/183 [==============================] - 40s 219ms/step - loss: 1.1745 - accuracy: 0.5740 - val_loss: 0.8247 - val_accuracy: 0.7315
Epoch 3/10
183/183 [==============================] - 40s 219ms/step - loss: 0.7166 - accuracy: 0.7668 - val_loss: 0.6695 - val_accuracy: 0.7753
Epoch 4/10
183/183 [==============================] - 40s 219ms/step - loss: 0.4951 - accuracy: 0.8404 - val_loss: 0.5110 - val_accuracy: 0.8466
Epoch 5/10
183/183 [==============================] - 40s 219ms/step - loss: 0.3629 - accuracy: 0.8896 - val_loss: 0.4965 - val_accuracy: 0.8329
Epoch 6/10
183/183 [==============================] - 40s 219ms/step - loss: 0.2533 - accuracy: 0.9243 - val_loss: 0.4444 - val_accuracy: 0.8699
Epoch 7/10
183/183 [==============================] - 40s 219ms/ste

In [ ]:
learner.validate(class_names=categories)

                      precision    recall  f1-score   support

Software Engineering       0.92      0.86      0.89       159
                DBMS       0.78      0.89      0.83       141
  Data Communication       0.91      0.88      0.89       147
      Cyber Security       0.87      0.86      0.87       139
    Operating System       0.87      0.85      0.86       144

            accuracy                           0.87       730
           macro avg       0.87      0.87      0.87       730
        weighted avg       0.87      0.87      0.87       730



array([[136,   8,   4,   5,   6],
       [  4, 126,   2,   5,   4],
       [  0,  10, 129,   3,   5],
       [  3,   8,   5, 120,   3],
       [  5,  10,   2,   5, 122]])

In [ ]:
save_path = '/kaggle/working/dis'
ktrain.get_predictor(learner.model, preproc=trans).save(save_path)

In [ ]:
# Import necessary packages
from sklearn.metrics import accuracy_score, classification_report
import ktrain

# Specify the file path for saving and loading the model in the Kaggle working directory
save_path = '/kaggle/working/dis'

# Load the predictor from the specified path
predictor = ktrain.load_predictor(save_path)

# Assuming you have your test data available as 'X_test' and 'y_test' in your Kaggle environment
# Make predictions on the test data using the loaded predictor
y_pred = predictor.predict(X_test)

# Calculate accuracy by comparing the predicted labels with true labels
accuracy = accuracy_score(y_test, y_pred)
print(f"Test data accuracy: {accuracy:.2%}")

# Generate classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print("Classification Report:")
print(report)

Test data accuracy: 86.44%
Classification Report:
                      precision    recall  f1-score   support

      Cyber Security       0.89      0.84      0.86       152
                DBMS       0.85      0.86      0.85       164
  Data Communication       0.90      0.90      0.90       148
    Operating System       0.83      0.86      0.84       132
Software Engineering       0.85      0.87      0.86       134

            accuracy                           0.86       730
           macro avg       0.86      0.86      0.86       730
        weighted avg       0.87      0.86      0.86       730



In [ ]:
# Import necessary packages
import ktrain
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from scipy import stats

# Load the predictors (models) from the specified directories
predictor_disbert = ktrain.load_predictor('/kaggle/working/dis')
predictor_mbert = ktrain.load_predictor('/kaggle/working/mbert')


# Assuming you have your test data available as 'X_test' and 'y_test'
# Make predictions on the test data using each of the loaded predictors
y_pred_disbert = predictor_disbert.predict(X_test)
y_pred_mbert = predictor_mbert.predict(X_test)

# Ensemble the predictions using majority voting
# Majority voting: For each sample, choose the most common prediction among the models
y_pred_ensemble = stats.mode([ y_pred_disbert, y_pred_mbert], axis=0)[0][0]

# Calculate accuracy by comparing the ensemble predictions with true labels
accuracy = accuracy_score(y_test, y_pred_ensemble)
print(f"Ensemble test data accuracy: {accuracy:.2%}")

# Generate classification report for the ensemble predictions
report = classification_report(y_test, y_pred_ensemble)

# Print the classification report
print("Ensemble classification report distilbert+mbert:")
print(report)

Ensemble test data accuracy: 88.36%
Ensemble classification report distilbert+mbert:
                      precision    recall  f1-score   support

      Cyber Security       0.86      0.91      0.88       152
                DBMS       0.86      0.91      0.88       164
  Data Communication       0.90      0.90      0.90       148
    Operating System       0.89      0.86      0.87       132
Software Engineering       0.92      0.84      0.88       134

            accuracy                           0.88       730
           macro avg       0.89      0.88      0.88       730
        weighted avg       0.88      0.88      0.88       730



In [ ]:
import ktrain
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from scipy import stats

# Load the predictors (models) from the specified directories
predictor_roberta = ktrain.load_predictor('/kaggle/working/roberta')
predictor_disbert = ktrain.load_predictor('/kaggle/working/dis')


# Assuming you have your test data available as 'X_test' and 'y_test'
# Make predictions on the test data using each of the loaded predictors
y_pred_roberta = predictor_roberta.predict(X_test)
y_pred_disbert = predictor_disbert.predict(X_test)

# Ensemble the predictions using majority voting
# Majority voting: For each sample, choose the most common prediction among the models
y_pred_ensemble = stats.mode([ y_pred_roberta, y_pred_disbert], axis=0)[0][0]

# Calculate accuracy by comparing the ensemble predictions with true labels
accuracy = accuracy_score(y_test, y_pred_ensemble)
print(f"Ensemble test data accuracy: {accuracy:.2%}")

# Generate classification report for the ensemble predictions
report = classification_report(y_test, y_pred_ensemble)

# Print the classification report
print("Ensemble classification report roberta+distil m bert:")
print(report)

Ensemble test data accuracy: 88.08%
Ensemble classification report roberta+distil m bert:
                      precision    recall  f1-score   support

      Cyber Security       0.87      0.93      0.90       152
                DBMS       0.83      0.88      0.86       164
  Data Communication       0.92      0.93      0.92       148
    Operating System       0.88      0.83      0.86       132
Software Engineering       0.92      0.82      0.87       134

            accuracy                           0.88       730
           macro avg       0.88      0.88      0.88       730
        weighted avg       0.88      0.88      0.88       730



In [ ]:
# Import necessary packages
import ktrain
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from scipy import stats

# Load the predictors (models) from the specified directories
predictor_disbert = ktrain.load_predictor('/kaggle/working/dis')
predictor_roberta = ktrain.load_predictor('/kaggle/working/roberta')
predictor_mbert = ktrain.load_predictor('/kaggle/working/mbert')

# Assuming you have your test data available as 'X_test' and 'y_test'
# Make predictions on the test data using each of the loaded predictors
y_pred_disbert = predictor_disbert.predict(X_test)
y_pred_roberta = predictor_roberta.predict(X_test)
y_pred_mbert = predictor_mbert.predict(X_test)

# Ensemble the predictions using majority voting
# Majority voting: For each sample, choose the most common prediction among the models
y_pred_ensemble = stats.mode([y_pred_disbert, y_pred_roberta, y_pred_mbert], axis=0)[0][0]

# Calculate accuracy by comparing the ensemble predictions with true labels
accuracy = accuracy_score(y_test, y_pred_ensemble)
print(f"Ensemble test data accuracy: {accuracy:.2%}")

# Generate classification report for the ensemble predictions
report = classification_report(y_test, y_pred_ensemble)

# Print the classification report
print("Ensemble classification report:")
print(report)

Ensemble test data accuracy: 90.00%
Ensemble classification report:
                      precision    recall  f1-score   support

      Cyber Security       0.89      0.93      0.91       152
                DBMS       0.90      0.91      0.91       164
  Data Communication       0.90      0.94      0.92       148
    Operating System       0.87      0.89      0.88       132
Software Engineering       0.94      0.82      0.88       134

            accuracy                           0.90       730
           macro avg       0.90      0.90      0.90       730
        weighted avg       0.90      0.90      0.90       730

